In [ ]:
import pandas as pd
import datetime, pytz
import vectorbt as vbt
import numpy as np
import matplotlib.pyplot as plt
import json

import talib
import pandas_ta
import time

from importlib import reload


import sys 
sys.path.append('../../modules/')

from forwardtest import ForwardTest
from vbt_strategies import Strategy as Strat

In [ ]:
def getBounds(df_day):
    dr_time = df_day[df_day.index.hour >= 9 and df_day.index.hour < 10 ]
    idrup = max(dr_time['Open'].max(), dr_time['Close'].max())
    idrdown = min(dr_time['Open'].min(), dr_time['Close'].min())
    drup = dr_time['High'].max()
    drdown = dr_time['Low'].min()
    return  idrup, idrdown, drup, drdown

def getExec(df_day):
    idrup, idrdown, drup, drdown = idrup, idrdown, drup, drdown
    dr_exec = df_day[df_day.index.hour >= 10 and df_day.index.hour < 10 ]
    bullish = 
def retirement(df_day):
    #9h -> 10h en 5mn pour calculer les bornes



ModuleNotFoundError: No module named 'tvDatafeed'

In [ ]:
class DRstrat():
    def __init__(self, df):
        self.ODR = [3,4, 8.5]
        self.RDR = [9.5, 10.5, 16]
        self.ADR = [19.5, 20.5, 2]

        self.df = df.reset_index()
    
    def sep_days(self):
        self.df['change'] = self.df['datetime'].apply(lambda x : 1 if x.hour == 2 and x.minute == 5 and x.second ==0 else 0)
        self.df['change'] = self.df['change'].cumsum()
        gb = self.df.groupby('change')  
        self.lst_days =  dict(tuple(self.df.groupby('change')))  #[x for _, x in gb.groups]

    def exec_dr(self, day, start_hour, end_hour , lines):
        ###get DR and IDR values
        start_range = int ((start_hour - 2)*12 - 1)
        end_range = int((end_hour-2)*12)
        df = day.iloc[start_range:end_range]
        end_lines = int((lines-2)*12)
        df_exec = day.iloc[end_range:end_lines]
        DR_high = df['high'].max()
        DR_low = df['low'].min()
        IDR_high =max(df['open'].max(), df['close'].max())
        IDR_low =min(df['open'].min(), df['close'].min())
        mid = (DR_high + DR_low)/2
        ##exec
        TP_long = 2*DR_high - mid
        TP_short = 2*DR_low - mid
        exec = False
        long = False
        short = False
        for id, row in df_exec.iterrows ():
            if not exec :
                if row['low'] <= DR_low :
                    short = True
                    exec = True

                elif row['high'] >= DR_high:
                    short = True 
                    exec = True
            
            elif short :
                if row['high'] >= mid :
                    return -1
                elif row['low'] <= TP_short:
                    return 1

            elif long :
                if row['high'] >= TP_long:
                    return 1
                elif row['low'] <= mid :
                    return -1
        
        if exec :
            row = df_exec.iloc[-1]
            if long :
                return (row['close']- DR_high)/(DR_high - mid)
            else :
                return (row['close']- DR_low)/(DR_low - mid)
        
        return 0
    def get_fvg(self, day):
        fvg_long = []
        fvg_short = []
        for i in range(2,len(day)):
            if day.iloc[i-2]['low'] > day.iloc[i]['high']:
                fvg_short.append((day.iloc[i-2]['high'], day.iloc[i]['high']))
            elif day.iloc[i-2]['high'] < day.iloc[i]['low']:
                fvg_long.append((day.iloc[i-2]['low'], day.iloc[i]['low']))
        return fvg_long, fvg_short

    def retirement(self, day, start_hour, end_hour, lines):
        ###get DR and IDR values
        start_range = int ((start_hour - 2)*12 - 1)
        end_range = int((end_hour-2)*12)
        df = day.iloc[start_range:end_range]
        end_lines = int((lines-2)*12)
        df_exec = day.iloc[end_range:end_lines]
        DR_high = df['high'].max()
        DR_low = df['low'].min()
        IDR_high =max(df['open'].max(), df['close'].max())
        IDR_low =min(df['open'].min(), df['close'].min())
        mid = (DR_high + DR_low)/2
        #fvg_long, fvg_short = self.get_fvg(df)

        ### execution
        setup = False
        exec = False
        touch_high = False
        touch_low = False
        for id, row in df_exec.iterrows():
            if not setup :
                if row['high'] >= IDR_high :
                    touch_high = True
                    setup = True
                elif row['low'] <= IDR_low:
                    touch_low = True
                    setup = True
            elif not exec :
                if touch_high :
                    if row['low'] <= IDR_low :
                        exec = True
                else :
                    if row['high'] >= IDR_high :
                        exec = True
            if exec :
                if touch_high :
                    if row['low']<= DR_low :
                        return -1
                    if row['high'] >= IDR_high:
                        return (IDR_high - IDR_low)/(IDR_low - DR_low)
                if touch_low :
                    if row['high']>= DR_high:
                        return -1
                    if row['low'] <= IDR_low:
                        return (IDR_low - IDR_high)/(IDR_high - DR_high)
        if exec :
            row = df_exec.iloc[-1]
            return (row['close'] - IDR_high)/(IDR_high - DR_high)
        else :
            return 0

    def exec_retirement(self):
        lst_returns = []
        for day in self.lst_days:
            for start, end, lines in [self.ODR, self.RDR, self.ADR]:
                r = self.retirement(self.lst_days[day], start, end, lines)
                lst_returns.append(r)
        return np.array(lst_returns)





    def exec(self):
        lst_returns = []
        for day in self.lst_days:
            for start, end, lines in [self.ODR, self.RDR, self.ADR]:
                r = self.exec_dr(self.lst_days[day], start, end, lines)
                lst_returns.append(r)
        return np.array(lst_returns)
